In [5]:
from typing import Literal

import ollama
from loguru import logger
from pydantic import BaseModel, Field


In [6]:
SYSTEM_PROMPT: str = "You are a helpful assistant. Answer extremely concisely."


In [7]:
class Message(BaseModel):
    """A single message in the conversation."""

    role: Literal["system", "user", "assistant"]
    content: str


class Conversation(BaseModel):
    """Manages a conversation with message history."""

    history: list[Message] = Field(
        default_factory=lambda: [Message(role="system", content=SYSTEM_PROMPT)]
    )
    model: str = "llama3.2"

    def ask(self, question: str) -> str:
        """Ask a question and get a response, maintaining conversation history."""
        self.history.append(Message(role="user", content=question))

        messages = [msg.model_dump() for msg in self.history]
        response = ollama.chat(model=self.model, messages=messages)
        answer = response["message"]["content"]

        self.history.append(Message(role="assistant", content=answer))
        return answer

    def reset(self) -> None:
        """Reset the conversation history."""
        self.history = [Message(role="system", content=SYSTEM_PROMPT)]

In [9]:
# Create a new conversation
logger.info("Creating a new conversation")
conversation = Conversation()

# Ask initial question
question = "What is Iceland?"
answer = conversation.ask(question)
logger.info(f"Question: {question}")
logger.success(f"Answer: {answer}")

# Ask a follow-up question
followup = "What is its population?"
answer = conversation.ask(followup)
logger.info(f"Follow-up: {followup}")
logger.success(f"Answer: {answer}")

# To start fresh, reset the conversation:
conversation.reset()
# Or create a new instance:
# conversation = Conversation()

2025-07-12 13:28:43.110 | INFO     | __main__:<module>:2 - Creating a new conversation
2025-07-12 13:28:43.939 | INFO     | __main__:<module>:8 - Question: What is Iceland?
2025-07-12 13:28:43.941 | SUCCESS  | __main__:<module>:9 - Answer: Iceland is an island nation in the North Atlantic, known for its stunning landscapes, geothermal hot springs, and unique culture.
2025-07-12 13:28:44.203 | INFO     | __main__:<module>:14 - Follow-up: What is its population?
2025-07-12 13:28:44.205 | SUCCESS  | __main__:<module>:15 - Answer: Approximately 368,000 people.
